In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test_df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

In [4]:
train_df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [5]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

stopwords_ = set(stopwords.words('english'))
len(stopwords_)

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


179

In [6]:
import re, string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
def standardize(text):
    pattern = r'\\u[0-9a-fA-F]{4}'
    text = re.sub(pattern, '', text)
    text = text.replace('\\n', ' ')
    text = re.sub(r'\d+', '', text)
    text = ''.join(char for char in text if char not in string.punctuation).lower().strip()
    text = ' '.join(word for word in text.split()  if word not in stopwords_)
    return text

def add_tokens(x,y):
    text = x + ' [sep] ' + y
    return text

In [8]:
def preprocess(data, train = False):
    le = LabelEncoder()
    ds = data.copy()
    ds = ds.drop('id', axis = 1)
    if train:
        ds = ds.drop(['model_a', 'model_b'], axis = 1)
    cols = ['prompt','response_a', 'response_b']
    for col in cols:
        ds[col] = ds[col].apply(standardize)
    return ds

train = preprocess(train_df, train=True)
test = preprocess(test_df, train=False)

In [9]:
target_cols = [col for col in train_df.columns if col.startswith('winner')]

In [10]:
x = train.drop(target_cols, axis = 1)
y = train[target_cols]

In [11]:
x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size = 0.2, random_state = 42) 

In [12]:
x_test = test

In [13]:
train_combined = train['prompt'] + ' [SEP] ' + train['response_a'] + ' [SEP] ' + train['response_b'] 

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

2024-07-12 13:42:58.156425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 13:42:58.156532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 13:42:58.273658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
text_vectorization = layers.TextVectorization(
    max_tokens = 360000,
    output_mode = 'int',
    output_sequence_length = 1000,
)

text_vectorization.adapt(train_combined)

In [16]:
normalizer_xa = layers.Normalization()
normalizer_xb = layers.Normalization()

In [17]:
train_xa = x_tr['prompt'] + ' [SEP] ' + x_tr['response_a']
train_xb = x_tr['prompt'] + ' [SEP] ' + x_tr['response_b']
train_xa = text_vectorization(train_xa)
train_xb = text_vectorization(train_xb)

normalizer_xa.adapt(train_xa)
normalizer_xb.adapt(train_xb)

train_y = y_tr
train_data = {'prompt_a' : normalizer_xa(train_xa), 'prompt_b': normalizer_xb(train_xb)}

train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_y)).batch(32).prefetch(tf.data.AUTOTUNE)

In [18]:
val_xa = x_val['prompt'] + ' [SEP] ' + x_val['response_a']
val_xb = x_val['prompt'] + ' [SEP] ' + x_val['response_b']
val_xa = text_vectorization(val_xa)
val_xb = text_vectorization(val_xb)

val_y = y_val
val_data = {'prompt_a' : normalizer_xa(val_xa), 'prompt_b': normalizer_xb(val_xb)}

validation_ds = tf.data.Dataset.from_tensor_slices((val_data, val_y)).batch(32).prefetch(tf.data.AUTOTUNE)

In [19]:
test_xa = x_test['prompt'] + ' [SEP] ' + x_test['response_a']
test_xb = x_test['prompt'] + ' [SEP] ' + x_test['response_b']
test_xa = text_vectorization(test_xa)
test_xb = text_vectorization(test_xb)

test_data = {'prompt_a' : normalizer_xa(test_xa), 'prompt_b': normalizer_xb(test_xb)}

test_ds = tf.data.Dataset.from_tensor_slices((test_data)).batch(32).prefetch(tf.data.AUTOTUNE)

In [20]:
train_ds, validation_ds, test_ds

(<_PrefetchDataset element_spec=({'prompt_a': TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), 'prompt_b': TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)}, TensorSpec(shape=(None, 3), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=({'prompt_a': TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), 'prompt_b': TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)}, TensorSpec(shape=(None, 3), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec={'prompt_a': TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), 'prompt_b': TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)}>)

In [21]:
for batch in train_ds.take(1):
    features, labels = batch
    print(features['prompt_a'].shape, features['prompt_b'].shape)
    print(labels.shape)
    break

(32, 1000) (32, 1000)
(32, 3)


In [22]:
def get_model(learning_rate = 1e-3):
    inputs_a = keras.Input(shape = (1000,), name = 'prompt_a')
    inputs_b = keras.Input(shape = (1000,), name = 'prompt_b')
    xa = layers.Dense(512, activation = 'relu')(inputs_a)
    xb = layers.Dense(512, activation = 'relu')(inputs_b)
    xa = layers.Dropout(0.5)(xa)
    xb = layers.Dropout(0.5)(xb)
    xa = layers.Dense(256, activation = 'relu')(xa)
    xb = layers.Dense(256, activation = 'relu')(xb)
    xa = layers.Dropout(0.2)(xa)
    xb = layers.Dropout(0.2)(xb)
    xa = layers.Dense(128, activation = 'relu')(xa)
    xb = layers.Dense(128, activation = 'relu')(xb)
    xa = layers.Dropout(0.1)(xa)
    xb = layers.Dropout(0.1)(xb)
    xa = layers.Dense(64, activation = 'relu')(xa)
    xb = layers.Dense(64, activation = 'relu')(xb)
    com = layers.Concatenate(axis = -1)([xa, xb])
    outputs = layers.Dense(3, activation = 'softmax')(com)
    
    
    model = keras.Model(inputs = [inputs_a, inputs_b], outputs = outputs)
    model.compile(
        optimizer= keras.optimizers.RMSprop(learning_rate = learning_rate),
        loss = 'categorical_crossentropy'
    )
    
    return model


In [23]:
model = get_model(learning_rate = 1e-5)

history = model.fit(train_ds, epochs = 40, validation_data = validation_ds)

Epoch 1/40
  62/1437 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2509

I0000 00:00:1720791814.704552      74 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720791814.722629      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1437/1437 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1955

W0000 00:00:1720791826.849998      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1720791827.378999      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1437/1437 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - loss: 1.1955 - val_loss: 1.1114
Epoch 2/40
  72/1437 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.1451

W0000 00:00:1720791828.367282      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1548 - val_loss: 1.1010
Epoch 3/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1318 - val_loss: 1.0961
Epoch 4/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1231 - val_loss: 1.0923
Epoch 5/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1193 - val_loss: 1.0882
Epoch 6/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1126 - val_loss: 1.0858
Epoch 7/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 1.1079 - val_loss: 1.0837
Epoch 8/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1064 - val_loss: 1.0820
Epoch 9/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.1032 - val_loss: 1.0808
Epoch 10/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.0990 - val_loss: 1.0791
Epoch 11/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.0960 - val_loss: 1.0778
Epoch 12/40
1437/1437 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1.0924 - val_loss: 1.0768
Epoch 13/40
1437/1437 ━━━━━━━━━━━━━━━━━━

In [24]:
preds = model.predict(test_ds)
preds

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step


W0000 00:00:1720791987.306715      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


array([[0.311548  , 0.31236297, 0.37608904],
       [0.33616966, 0.35553536, 0.30829498],
       [0.29367808, 0.40294108, 0.30338082]], dtype=float32)

In [25]:
sub = pd.DataFrame(preds, columns = target_cols)
sub.insert(0, 'id', test_df['id'])
sub.to_csv('submission.csv', index = False)